In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial
import pandas as pd
from tqdm.autonotebook import tqdm
import os
import matplotlib.pyplot as plt
import json
import nltk
import re
import concurrent.futures
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import OrderedDict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import nltk
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp.max_length = 4000000
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS

from tqdm.autonotebook import tqdm
import string

%matplotlib inline

os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')

['sample_submission.csv', 'train.csv', 'test', 'train']

In [2]:
# reading csv files and train & test file paths
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [4]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [5]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [6]:
%%time
tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

  0%|          | 0/19661 [00:00<?, ?it/s]

CPU times: user 11.2 s, sys: 5.21 s, total: 16.5 s
Wall time: 1min 6s


In [7]:
%%time
tqdm.pandas()
sample_sub['text'] = sample_sub['Id'].progress_apply(partial(read_append_return, train_files_path=test_files_path))

  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 41.4 ms, sys: 3.67 ms, total: 45.1 ms
Wall time: 55.9 ms


In [8]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
#     text = re.sub("/'+/g", ' ', text)
    
    return text

In [9]:
%%time
tqdm.pandas()
train_df['text'] = train_df['text'].progress_apply(text_cleaning)

  0%|          | 0/19661 [00:00<?, ?it/s]

CPU times: user 3min 38s, sys: 5.84 s, total: 3min 43s
Wall time: 3min 42s


In [10]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [11]:
train_df.head(4)

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
0,8d61b2de-3b1e-4232-863b-9ba342d9ad65,Public School Choice and Racial Sorting: An Ex...,Common Core of Data,NCES Common Core of Data,nces common core of data,there has been a longstanding concern among ed...
1,cf0223e8-f25e-4ea4-b63e-e971d72a9caa,Determining the Effects of Pre-College STEM Co...,Education Longitudinal Study,Education Longitudinal Study,education longitudinal study,many stem studies have focused on traditional ...
2,85b70b11-8895-456b-b133-df89b0bbff8a,Data Augmentation Using Learned Transformation...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni,image segmentation is an important task in man...
3,b21c09ca-2eeb-499c-9e73-9103871abafd,Vestibular Loss in Older Adults Is Associated ...,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging,baltimore longitudinal study of aging,background vestibular inputs have been shown t...


In [12]:
sample_sub

,Id,PredictionString,text
0,2100032a-7c33-4bff-97ef-690822c43466,NaN,Cognitive deficits and reduced educational ach...
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN,This report describes how the education system...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN,"Cape Hatteras National Seashore (CAHA), locate..."
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN,A significant body of research has been conduc...


In [13]:
Train_df = train_df.copy()

In [14]:
Train_df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
0,8d61b2de-3b1e-4232-863b-9ba342d9ad65,Public School Choice and Racial Sorting: An Ex...,Common Core of Data,NCES Common Core of Data,nces common core of data,there has been a longstanding concern among ed...
1,cf0223e8-f25e-4ea4-b63e-e971d72a9caa,Determining the Effects of Pre-College STEM Co...,Education Longitudinal Study,Education Longitudinal Study,education longitudinal study,many stem studies have focused on traditional ...
2,85b70b11-8895-456b-b133-df89b0bbff8a,Data Augmentation Using Learned Transformation...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni,image segmentation is an important task in man...
3,b21c09ca-2eeb-499c-9e73-9103871abafd,Vestibular Loss in Older Adults Is Associated ...,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging,baltimore longitudinal study of aging,background vestibular inputs have been shown t...
4,64488cc2-7515-4385-a3b7-64c50767d526,Prediction of Incipient Alzheimer's Disease De...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni,background mild cognitive impairment mci is a ...


In [15]:
text = Train_df["text"].tolist()
testtext = sample_sub['text'].tolist()

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
text2vec = []
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
for i in range(0,1):
    vec = vectorizer.transform((text[i])) 
    text2vec.append(np.asarray(vec))

ValueError: Iterable over raw text documents expected, string object received.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
test_text2vec  = [vectorizer.transform((testtext[i])) 
            for i in range(0,len(Train_df['text']))]

In [ ]:
sparce_matrix = text2vec
y = Train_df.iloc[:,2].values 
X = sparce_matrix

In [ ]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
decoded = encoder.inverse_transform(y)
decoded[:10]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=45)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc1=RandomForestClassifier(n_estimators= 50, max_depth=15, bootstrap=True, random_state=45)
rfc1.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = rfc1.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
# print('Confusion matrix:\n',confusion_matrix(y_test,y_pred))
# print('Classification report:\n',classification_report(y_test,y_pred))

In [ ]:
def top_n_pred_prob_df(n, model, X_test, column_name):
  predictions = model.predict_proba(X_test)
  preds_idx = np.argsort(-predictions) 
  classes = pd.DataFrame(model.classes_, columns=['class_name'])
  classes.reset_index(inplace=True)
  top_n_preds = pd.DataFrame()
  for i in range(n):
    top_n_preds[column_name + 'Prediction {}_num'.format(i)] =     [preds_idx[doc][i] for doc in range(len(X_test))]
    top_n_preds[column_name + 'Prediction {} probability'.format(i)] = [predictions[doc][preds_idx[doc][i]] for doc in range(len(X_test))]
    top_n_preds = top_n_preds.merge(classes, how='left', left_on= column_name + 'Prediction {}_num'.format(i), right_on='index')
    top_n_preds = top_n_preds.rename(columns={'class_name': column_name + 'Prediction {}'.format(i)})
    try: top_n_preds.drop(columns=['index', column_name + 'Prediction {}_num'.format(i)], inplace=True) 
    except: pass
  return top_n_preds.iloc[0]

In [ ]:
Prediction_df = pd.DataFrame()

for i in range (4):
    x = top_n_pred_prob_df(10,rfc1,np.array(test_text2vec[i]).reshape(1, -1), "")
    Prediction_df = Prediction_df.append(x, ignore_index=True)

In [ ]:
Prediction_df

In [ ]:
predicts = []
for i in range (4):
    lol  = np.array(test_text2vec[i]).reshape(1, -1)
    pred_test =  rfc1.predict(lol)
    X = encoder.inverse_transform(pred_test)
    X = X.tolist()[0]
    predicts.append(X)
predicts

In [ ]:
predicts1 = []
predicts2 = []
predicts3 = []
predicts4 = []
predicts5 = []

for i in range (4):
    X = int(Prediction_df.iloc[i]['Prediction 0'])
    newArray = np.append (0, X)
    newArray = np.delete(newArray, 0)
    Y = encoder.inverse_transform(newArray)
    Y = Y.tolist()[0]
    predicts1.append(Y)
    X = int(Prediction_df.iloc[i]['Prediction 1'])
    newArray = np.append (0, X)
    newArray = np.delete(newArray, 0)
    Y = encoder.inverse_transform(newArray)
    Y = Y.tolist()[0]
    predicts2.append(Y)
    X = int(Prediction_df.iloc[i]['Prediction 2'])
    newArray = np.append (0, X)
    newArray = np.delete(newArray, 0)
    Y = encoder.inverse_transform(newArray)
    Y = Y.tolist()[0]
    predicts3.append(Y)
    X = int(Prediction_df.iloc[i]['Prediction 3'])
    newArray = np.append (0, X)
    newArray = np.delete(newArray, 0)
    Y = encoder.inverse_transform(newArray)
    Y = Y.tolist()[0]
    predicts4.append(Y)
    X = int(Prediction_df.iloc[i]['Prediction 4'])
    newArray = np.append (0, X)
    newArray = np.delete(newArray, 0)
    Y = encoder.inverse_transform(newArray)
    Y = Y.tolist()[0]
    predicts5.append(Y)


In [ ]:
temp1 = [x.lower() for x in predicts1]
temp2 = [x.lower() for x in predicts2]
temp3 = [x.lower() for x in predicts3]
temp4 = [x.lower() for x in predicts4]
temp5 = [x.lower() for x in predicts5]


existing_labels = (temp1 + temp2 + temp3 + temp4 + temp5)
id_list = []
lables_list = []
for index, row in tqdm(sample_sub.iterrows()):
    sample_text = row['text']
    row_id = row['Id']
    temp_df = train_df[train_df['text'] == text_cleaning(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    for known_label in existing_labels:
        if known_label in sample_text.lower():
            cleaned_labels.append(clean_text(known_label))
    cleaned_labels = [clean_text(x) for x in cleaned_labels]
    cleaned_labels = set(cleaned_labels)
    lables_list.append('|'.join(cleaned_labels))
    id_list.append(row_id)

In [ ]:
sample_subf = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
sample_subf.PredictionString = lables_list
sample_subf.to_csv('./submission.csv', index=False)

In [ ]:
sample_subf.head()